![Urgences - Image CC0 - pexels.com](img/pexels-pixabay-263402.jpg "Urgences")

# Challenge - [ED Lab Prediction]
_Nom à trouver_

## Objectif

Ce notebook effectue le pre-processing des données.  
Il exploite les données stockées dans la base sqlite, téléchargées à partir de `download_data.py` et les exporte dans un fichier csv à destination de l'entrainement de la data-visualisation l'algorithme.

## Chargement des données

In [4]:
import sqlite3
import pandas as pd

In [110]:
# Sqlite connection
conn = sqlite3.connect("./data/mimic-iv.sqlite")

# Custom lab items classification
items = pd.read_csv("./config/lab_items.csv").dropna()
items_list = items["item_id"].astype("str").tolist()

In [111]:
# Création d'un index pour accélérer les requêtes
conn.execute("CREATE INDEX IF NOT EXISTS biological_index ON labevents (stay_id, itemid)");

## Stays dataset

In [148]:
stays = pd.read_sql(f"""
    SELECT 
        s.stay_id,
        s.intime intime,
        p.gender gender,
        p.anchor_age age,
        t.temperature,
        t.heartrate,
        t.resprate,
        t.o2sat,
        t.sbp,
        t.dbp,
        t.pain,
        t.chiefcomplaint
    FROM edstays s
    LEFT JOIN patients p
        ON p.subject_id = s.subject_id
    LEFT Join triage t
        ON t.stay_id = s.stay_id
""", conn)

In [151]:
# @TODO : include drugs

In [178]:
# Writting
stays.to_csv("./data/features.csv")

## Lab dataset

In [112]:
labs = pd.read_sql(f"""
    SELECT 
        le.stay_id,
        le.itemid item_id
    FROM labevents le
    WHERE le.itemid IN ('{"','".join(items_list)}')
    GROUP BY
        le.stay_id,
        le.itemid
""", conn)

In [119]:
labs_deduplicate = pd.merge(
    items[["item_id","3"]].rename(columns={"3":"label"}),
    labs,
    left_on="item_id",
    right_on="item_id"
).drop_duplicates(["stay_id", "label"])[["stay_id","label"]] \
 .reset_index(drop=True)

In [170]:
labs_deduplicate_pivot = pd.pivot_table(
    labs_deduplicate.assign(value=1),
    index=["stay_id"],
    columns=["label"],
    values="value"
).fillna(0)

In [176]:
labs_deduplicate_pivot_final = labs_deduplicate_pivot.join(
    stays[["stay_id"]].set_index("stay_id"),
    how="right"
).fillna(0).astype("int8").reset_index()

In [179]:
labs_deduplicate_pivot_final.to_csv("./data/labels.csv")